In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib as mpl
import h5py
import tqdm


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

import sys
sys.path.append('/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation')
import anatomy as anat

In [ ]:
file_path = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss63_09Bparams_2023-06-22_uniform-09b_n500_r100_f40_l10'
hdf = h5py.File(file_path+'/sam_lib.hdf5', 'r')

print(hdf.keys())
sample_params = hdf['sample_params'][...]
hc_ss = hdf['hc_ss'][...]
hc_bg = hdf['hc_bg'][...]
sspar = hdf['sspar'][...]
bgpar = hdf['bgpar'][...]
fobs_gw_cents = hdf['fobs'][:]
hdf.close()

dur = 1/fobs_gw_cents[0]
fobs_gw_cents, fobs_gw_edges = holo.utils.pta_freqs(dur, num=len(fobs_gw_cents))
nsamps, nfreqs, nreals, nloudest = [*hc_ss.shape]


In [ ]:
npz = np.load(file_path+'/PS_Uniform_09B.pspace.npz', allow_pickle=True)
print(npz.files)
param_names = npz['param_names']
print(param_names)
print(npz['class_name'])
lib_name = '%s v%s, %d samples' % (npz['class_name'], npz['librarian_version'], nsamps)
print(lib_name)
# print(npz['sam_shape'])
npz.close()

In [ ]:
# Load detstats
npz = np.load('/Users/emigardiner/GWs/holodeck/output/brc_output/ss63_09Bparams_2023-06-22_uniform-09b_n500_r100_f40_l10/detstats/clbrt_psrs40/detstats_lib.npz')
print(npz.files)
# print(npz['snr_ss'].shape)
dp_ss = npz['dp_ss']
dp_bg = npz['dp_bg']
df_ss = npz['df_ss']
df_bg = npz['df_bg']
ev_ss = npz['ev_ss']
nskies = dp_ss.shape[-1]
print(f"{ev_ss.shape=}, {dp_ss.shape=}")
npz.close()

In [ ]:
print(sample_params.shape
      )

# Get Edges

In [ ]:
NBINS = 40
ht_min = np.min(sample_params[:,0])
ht_max = np.max(sample_params[:,0])
print(f"{ht_min=}, {ht_max=}")
ht_edges = np.linspace(ht_min, ht_max, NBINS)
ev_edges = np.geomspace(2.5e-3, 10.0, NBINS)

# Get data

In [ ]:

ev2dp = ev_ss/dp_bg[...,np.newaxis]
thard = sample_params[:,0]
print(f"{ev2dp.shape=}, {thard.shape=}")
thard = np.repeat(thard, nreals*nskies).reshape(nsamps, nreals, nskies)
print(f"{ev2dp.shape=}, {thard.shape=}")


# Make Histogram

In [ ]:
hist, eve, hte = np.histogram2d(ev2dp.flatten(), thard.flatten(), 
                            bins=(ev_edges, ht_edges))


In [ ]:
xlabel = plot.PARAM_KEYS['hard_time']
ylabel = '$EV_\mathrm{SS}/DP_\mathrm{BG}$'
xscale = 'linear'
yscale = 'log'


htgrid, evgrid = np.meshgrid(ht_edges, ev_edges)
fig, ax = plot.figax_single(xscale=xscale, yscale=yscale, xlabel=xlabel, ylabel=ylabel)
im = ax.pcolormesh(htgrid, evgrid, np.log10(hist))
plt.colorbar(im, ax=ax, label='$\log N$')

# For Every Parameter

In [ ]:
NBINS = 40

yy = ev_ss/dp_bg[...,np.newaxis]
yy_edges = np.geomspace(2.5e-4, 15.0, NBINS)

param_hists = []
param_edges = []
for ii, target in enumerate(param_names):
    xx = sample_params[:,ii]
    xx = np.repeat(xx, nreals*nskies)
    xmin = np.min(xx)
    xmax = np.max(xx)
    xx_edges = np.linspace(xmin, xmax, NBINS)
    print(f"{target} {xmin=}, {xmax=}")

    hist, yye, xxe = np.histogram2d(yy.flatten(), xx, bins=(yy_edges, xx_edges))
    param_hists.append(hist)
    param_edges.append(xx_edges)



In [ ]:
fig, axs = plot.figax_double(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear')
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[param_names[ii]])
    if ii == 0 or ii == 3:
        ax.set_ylabel(ylabel)
    
    xxgrid, yygrid = np.meshgrid(param_edges[ii], yy_edges)
    im = ax.pcolormesh(xxgrid, yygrid, param_hists[ii])

plt.subplots_adjust(wspace=0.03)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.01, label='$N$', )

In [ ]:
fig, axs = plot.figax_double(nrows=2, ncols=3, sharey=True, sharex=False, xscale='linear')
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[param_names[ii]])
    if ii == 0 or ii == 3:
        ax.set_ylabel(ylabel)
    
    xxgrid, yygrid = np.meshgrid(param_edges[ii], yy_edges)
    im = ax.pcolormesh(xxgrid, yygrid, np.log10(param_hists[ii]))

plt.subplots_adjust(wspace=0.03)

# add colorbar
plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.7, 
             orientation='vertical', pad=0.01, label='$\log N$', )